# Data Science for Business - GAM on Ames Housing

## Initialize notebook
Load required packages. Set up workspace, e.g., set theme for plotting and initialize the random number generator.

In [ ]:
# Install packages that are not already installed on Colab
#!pip install pygam

In [ ]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error

import statsmodels.formula.api as smf

import pygam
from pygam import LinearGAM, s, l, f, te

In [ ]:
plt.style.use('fivethirtyeight')

## Problem description

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence. With 76 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this dataset challenges you to predict the final price of each home. More: <https://www.kaggle.com/c/house-prices-advanced-regression-techniques>


## Load data

Load data from CSV file.

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/olivermueller/ds4b-2024/refs/heads/main/Session_06/ameshousing.csv')

In [ ]:
data.head()

## Prepare data

Let us focus on the numerical variables only.   

In [ ]:
col_num = data.select_dtypes(include=[np.number])
data = data[col_num.columns]

In [ ]:
data.head()

Finally, we will split the data into features (*X*) and labels (*y*) and into training (*X_train, y_train*) and test (*X_test, y_test*) sets.

In [ ]:
X = data.drop("SalePrice", axis=1)
y = data["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

In [ ]:
X_train.columns

## Fit a Linear Regression as the Baseline

To assess the performance of the GAM, we first fit a simple baseline model. Specifically, a linear regression model with two predictors.

In [ ]:
mod_01 = smf.ols(formula='SalePrice ~ LotArea + GrLivArea', data=pd.concat([X_train, y_train], axis=1))
mod_01 = mod_01.fit()
print(mod_01.summary())

Make predictions and caclulate the RMSE.

In [ ]:
y_pred = mod_01.predict(X_test)
root_mean_squared_error(y_test, y_pred)

## Fit a GAM

The following code illustrates how to fit a GAM with two smoothing spline with a 2nd derivative smoothness constraint and two cubic terms.

Unfortunateley, the API is not very user friendly. The features have to be specified by using their column index (e.g., s(0, spline_order=3) refers to the first column of the feature matrix (LotArea)).

In [ ]:
gam_mod = LinearGAM(s(2, spline_order=3) + s(15, spline_order=3))
gam_mod.fit(X_train, y_train)

Let's have a look at the model. Due to the non-linearity, the model is not easy to interpret. But we can extract some useful statistics about model fit from the summary (e.g., R^2, GCV error).

In [ ]:
gam_mod.summary()

Calculate RMSE from GCSV error.

In [ ]:
np.sqrt(gam_mod.statistics_['GCV'])

We can see that the model fit on both training data (R2) and test data (RMSE) is better than with the linear regression model.

Let's try to interpret the patterns the model has learned. For splines and other non-linear models we typically do that in a visual way. In the following, we will create partial dependence plots (incl. confidence intervals) for all terms of the model. 

In [ ]:
for i, term in enumerate(gam_mod.terms):
    if term.isintercept:
        continue

    x_grid = gam_mod.generate_X_grid(term=i)
    pdep, confi = gam_mod.partial_dependence(term=i, X=x_grid, width=0.95)

    feature_name = X_train.columns[term.feature]
    print(feature_name)

    plt.figure()
    plt.plot(x_grid[:, term.feature], pdep)
    plt.plot(x_grid[:, term.feature], confi, c='r', ls='--')
    plt.title(repr(term))
    plt.show()

## Your Turn!

Pick two other variables from the dataset and try model their relationship with the house price using a GAM.

In [ ]:
# YOUR CODE GOES HERE

## Tune the Smoothness Penalty

For fitting the model above, we have used the default value for the smoothing parameter lambda. 

In [ ]:
print(gam_mod.lam)

Let’s perform a grid-search over multiple lambda values to see if we can improve our model. We will search for the model with the lowest generalized cross-validation (GCV) score. Let’s try 20 values for each smoothing parameter, resulting in a total of 20*20 = 100 points in our grid.

In [ ]:
lam = np.logspace(-10, 10, 20)
lams = [lam] * 2

gam_mod.gridsearch(X_train, y_train, lam=lams)


In [ ]:
gam_mod.summary()

Calculate the RMSE based on the GCV error. As we can see, the tuning improved our model slightly.

In [ ]:
np.sqrt(gam_mod.statistics_['GCV'])

Finally, let's visualize the effect of the tuning on the learned relationships.

In [ ]:
for i, term in enumerate(gam_mod.terms):
    if term.isintercept:
        continue

    x_grid = gam_mod.generate_X_grid(term=i)
    pdep, confi = gam_mod.partial_dependence(term=i, X=x_grid, width=0.95)

    plt.figure()
    plt.plot(x_grid[:, term.feature], pdep)
    plt.plot(x_grid[:, term.feature], confi, c='r', ls='--')
    plt.title(repr(term))
    plt.show()